In [1]:
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
dataset_name = "HuggingFaceH4/MATH-500"

In [2]:
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

prompt = "How many r in raspberry"
messages = [
    {"role": "system", "content": "You are a helpful and harmless assistant. You are DeepSeek-R1. You should think step-by-step."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=8192,
    output_hidden_states=True
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

/home/xzascc/Documents/code/ReflectionDirection/.venv/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:818: UserWarning: `return_dict_in_generate` is NOT set to `True`, but `output_hidden_states` is. When `return_dict_in_generate` is not `True`, `output_hidden_states` is ignored.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


In [7]:
print(response)

Okay, so I'm trying to figure out how many times the letter 'r' appears in the word "Raspberry." Let me break this down step by step.

First, I'll write out the word: Raspberry. Now, I need to go through each letter one by one and count how many 'r's there are. Sometimes, people get confused with similar-sounding letters, so I should make sure I don't miss any.

Starting from the beginning: R. That's one 'r'.

Next, A. No 'r' here.

Then, P. Still nothing.

F. Nope.

R. That's the second 'r'.

A. Again, no luck.

S. No.

I. No.

C. No.

E. No.

So, after going through each letter, I only found one 'r' in the word. Let me double-check to make sure I didn't miss anything. Maybe I should write it down again: R-A-P-F-R-A-S-I-C-E. Yep, only one 'r' at the beginning.

I think that's it. So, the answer is just one 'r' in "Raspberry."
</think>

The word "Raspberry" contains only one 'r'. 

**Answer:** There is 1 'r' in Raspberry.


In [3]:
import torch

class HookedModel(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
        self.hidden_states = []

    def hook_fn(self, module, input, output):
        self.hidden_states.append(output.clone().detach())

    def forward(self, *args, **kwargs):
        # Register hooks on all transformer blocks
        for layer in self.model.model.layers:
            layer.register_forward_hook(self.hook_fn)

        # Run the model
        output = self.model(*args, **kwargs)
        return output, self.hidden_states

In [4]:
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer

model = LLM(model_name)
tok = AutoTokenizer.from_pretrained(model_name)
hooked_model = HookedModel(model)
stop_token_ids = tok("<|im_end|>")["input_ids"]

sampling_params = SamplingParams(
    max_tokens=8192,
    min_tokens=0,
    stop_token_ids=stop_token_ids,
)
model = LLM(model_name, max_model_len=100000)
prompt = "How many r in raspberry"
prompt = (
    "<|im_start|>system\nYou are DeepSeek-R1, created by DeepSeek. You are a helpful assistant.<|im_end|>\n<|im_start|>user\n"
    + prompt
    + "<|im_end|>\n<|im_start|>assistant\n"
)

output, hidden_states = hooked_model.generate(prompt, sampling_params=sampling_params)

INFO 02-19 22:19:18 __init__.py:190] Automatically detected platform cuda.
INFO 02-19 22:19:50 config.py:542] This model supports multiple tasks: {'embed', 'generate', 'score', 'classify', 'reward'}. Defaulting to 'generate'.
WARNING 02-19 22:19:50 arg_utils.py:1135] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 02-19 22:19:50 config.py:1556] Chunked prefill is enabled with max_num_batched_tokens=2048.
INFO 02-19 22:19:50 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B', speculative_config=None, tokenizer='deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, 

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 02-19 22:19:56 model_runner.py:1115] Loading model weights took 3.3460 GB
INFO 02-19 22:19:57 worker.py:267] Memory profiling takes 0.79 seconds
INFO 02-19 22:19:57 worker.py:267] the current vLLM instance can use total_gpu_memory (23.99GiB) x gpu_memory_utilization (0.90) = 21.59GiB
INFO 02-19 22:19:57 worker.py:267] model weights take 3.35GiB; non_torch_memory takes 0.08GiB; PyTorch activation peak memory takes 1.39GiB; the rest of the memory reserved for KV Cache is 16.78GiB.
INFO 02-19 22:19:57 executor_base.py:110] # CUDA blocks: 39264, # CPU blocks: 9362
INFO 02-19 22:19:57 executor_base.py:115] Maximum concurrency for 131072 tokens per request: 4.79x
INFO 02-19 22:19:57 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_u

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:09<00:00,  3.68it/s]

INFO 02-19 22:20:07 model_runner.py:1562] Graph capturing finished in 10 secs, took 0.23 GiB
INFO 02-19 22:20:07 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 10.87 seconds


...

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 02-19 22:20:11 model_runner.py:1115] Loading model weights took 3.3148 GB
INFO 02-19 22:20:26 worker.py:267] Memory profiling takes 14.58 seconds
INFO 02-19 22:20:26 worker.py:267] the current vLLM instance can use total_gpu_memory (23.99GiB) x gpu_memory_utilization (0.90) = 21.59GiB
INFO 02-19 22:20:26 worker.py:267] model weights take 3.31GiB; non_torch_memory takes -1.90GiB; PyTorch activation peak memory takes 1.38GiB; the rest of the memory reserved for KV Cache is 18.79GiB.
INFO 02-19 22:20:26 executor_base.py:110] # CUDA blocks: 43979, # CPU blocks: 9362
INFO 02-19 22:20:26 executor_base.py:115] Maximum concurrency for 100000 tokens per request: 7.04x
INFO 02-19 22:20:26 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:33<00:00,  1.06it/s]

INFO 02-19 22:21:01 model_runner.py:1562] Graph capturing finished in 31 secs, took 0.00 GiB
INFO 02-19 22:21:01 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 50.24 seconds


AttributeError: 'HookedModel' object has no attribute 'generate'